本文档详细介绍了阿里巴巴团队研发的 Qwen2-VL 系列，这是 Qwen-VL 模型的全面升级版本，旨在提升视觉-语言任务的性能。Qwen2-VL 的核心创新包括引入动态分辨率机制和多模态旋转位置嵌入（M-RoPE）。动态分辨率机制允许模型处理任意分辨率的图像，动态生成视觉 token，并通过压缩邻近 token 减少序列长度，从而高效捕获多尺度信息。而 M-RoPE 将时间、高度和宽度的位置信息结合，显著增强模型对多模态输入（图像、视频和文本）的感知能力。此外，模型通过统一的图像和视频处理框架，结合混合训练策略，进一步提升了对静态和动态内容的理解。

模型采用三阶段训练策略：

 **1.** **第一阶段**：单独训练视觉 Transformer（ViT），重点学习图像与文本之间的语义关系。

 **2.** **第二阶段**：解冻所有参数，在更广泛的数据集上进行多模态综合训练。

 **3.** **第三阶段**：冻结 ViT 参数，专注于通过指令数据集微调语言模型，提升其任务执行能力。

Qwen2-VL 提供三种规模（2B、8B 和 72B 参数版本），能够适应不同硬件资源和任务需求。其多模态模型框架在视觉问答、文档理解、视频分析和多语言文本识别等任务中表现出色，尤其在处理长视频（超过20分钟）和多语言 OCR 任务时展现了卓越能力。通过分阶段训练和丰富的数据来源，模型实现了在多个视觉语言基准测试中的领先表现，并在任务复杂性和多模态理解的扩展性上设立了新的标杆。

Qwen2-VL 系列的关键特性还包括对视频的统一处理（每秒采样两帧，结合 3D 卷积处理帧序列）、动态调整输入分辨率以平衡性能和计算效率，以及对特定场景的功能调用能力优化。整体来看，该模型不仅在技术上推动了视觉语言模型的发展，还通过开源的方式支持了广泛的应用和研究场景，为多模态 AI 系统的未来发展提供了坚实基础。

![jupyter](https://camo.githubusercontent.com/a98a801cab554fd20099a2c3fcce4597f45866b31f5d78f391c91196f4a3e57e/68747470733a2f2f7169616e77656e2d7265732e6f73732d616363656c65726174652d6f766572736561732e616c6979756e63732e636f6d2f5177656e322d564c2f7177656e325f766c5f6672616d65776f726b2e6a7067)

### 图片来源:
https://github.com/QwenLM/Qwen2-VL

# Qwen2-VL多模态模型代码解析

## **1.模型主要模块**
Qwen2-VL 模型的核心模块包括 **视觉编码器**、**语言模型** 和 **多模态融合**，它们分别处理图像和文本数据，并结合两者的特征以完成任务。下面将详细介绍这些模块，并结合具体代码进行解析。

## **1.1 核心模块功能详解**

## 1.1.1 _get_unpad_data

**功能：** 用于处理批量序列数据的填充与解填充：

**输入：**attention_mask，标识序列中有效元素的二进制掩码。

**输出：**

  **有效索引：**去掉padding后的序列索引。

  **累计长度（cu_seqlens）：** 分隔不同序列的位置。

  **最大长度：** 为高效计算确定所需的最大序列长度。


In [ ]:
def _get_unpad_data(attention_mask):
    seqlens_in_batch = attention_mask.sum(axis=-1, dtype="int32")  # 计算每个序列的长度（非0的元素数量）。
    indices = paddle.nonzero(attention_mask.flatten(), as_tuple=False).flatten()  # 找出mask中非0元素的索引。
    max_seqlen_in_batch = seqlens_in_batch.max().item()  # 找出batch中最大序列长度。
    cu_seqlens = F.pad(paddle.cumsum(seqlens_in_batch, axis=0), (1, 0))  # 累加每个序列长度，用于unpadding。
    return (indices, cu_seqlens, max_seqlen_in_batch)


### 1.1.2    is_casual_mask

**功能：** 判断一个掩码矩阵是否是因果掩码（Causal Mask）：

因果掩码的特点是**只允许访问当前位置及之前的序列**，因此是一个**上三角矩阵**。

In [ ]:
def is_casual_mask(attention_mask):
    return (paddle.triu(attention_mask) == attention_mask).all().item()


### 1.1.3 _make_causal_mask

**功能：** 创建**因果掩码**，并支持增量注意力机制：

适用于自回归模型的注意力机制（防止未来时刻的信息泄露）。
支持结合过去的注意力历史（past_key_values_length），扩展到更大的维度。

In [ ]:
def _make_causal_mask(input_ids_shape, past_key_values_length):
    batch_size, target_length = input_ids_shape
    mask = paddle.tril(paddle.ones((target_length, target_length), dtype="bool"))  # 创建下三角矩阵。
    if past_key_values_length > 0:
        mask = paddle.concat([paddle.ones([target_length, past_key_values_length], dtype="bool"), mask], axis=-1)
    return mask[None, None, :, :].expand([batch_size, 1, target_length, target_length + past_key_values_length])


### 1.1.4 _expand_2d_mask

**功能：** 将一个简单的二维掩码扩展为适合注意力计算的4维掩码：

输入：[batch_size, src_length]。
输出：[batch_size, 1, tgt_length, src_length]，为注意力模块准备的输入。

In [ ]:
def _expand_2d_mask(mask, dtype, tgt_length):
    batch_size, src_length = mask.shape[0], mask.shape[-1]
    tgt_length = tgt_length if tgt_length is not None else src_length
    mask = mask[:, None, None, :].astype("bool")  # 增加维度，适配注意力计算。
    mask.stop_gradient = True
    expanded_mask = mask.expand([batch_size, 1, tgt_length, src_length])
    return expanded_mask


### 1.1.5 Qwen2VLCausalLMOutputWithPast

这是一个输出类，专为 Qwen2-VL 模型生成语言模型输出设计，继承了 ModelOutput 基类，包含以下字段：

**loss:** 可选的语言建模损失，用于训练时评估预测的准确性。

**logits:** 模型生成的预测分数，维度为 (batch_size, sequence_length, vocab_size)，表示每个时间步的词汇预测分数。

**past_key_values:** 用于缓存历史计算的注意力 key 和 value，可加速序列生成。

**hidden_states:** 存储每一层的隐藏状态，可用于进一步分析模型内部表示。

**attentions:** 存储注意力权重，用于理解模型的注意力分配。

**rope_deltas:** 表示序列长度和多模态旋转位置编码之间的索引差值。

**用途：**为 Qwen2-VL 模型在推理和训练阶段返回丰富的信息。

In [ ]:
class Qwen2VLCausalLMOutputWithPast(ModelOutput):
    """
    Base class for Qwen2VL causal language model (or autoregressive) outputs.

    Args:
        loss (`paddle.Tensor` of shape `(1,)`, *optional*, returned when `labels` is provided):
            Language modeling loss (for next-token prediction).
        logits (`paddle.Tensor` of shape `(batch_size, sequence_length, config.vocab_size)`):
            Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
        past_key_values (`tuple(tuple(paddle.Tensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(paddle.Tensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`)

            Contains pre-computed hidden-states (key and values in the self-attention blocks) that can be used (see
            `past_key_values` input) to speed up sequential decoding.
        hidden_states (`tuple(paddle.Tensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `paddle.Tensor` (one for the output of the embeddings, if the model has an embedding layer, +
            one for the output of each layer) of shape `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.
        attentions (`tuple(paddle.Tensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `paddle.Tensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
        rope_deltas (`paddle.Tensor` of shape `(batch_size, )`, *optional*):
            The rope index difference between sequence length and multimodal rope.
    """

    loss: Optional[paddle.Tensor] = None
    logits: paddle.Tensor = None
    past_key_values: Optional[List[paddle.Tensor]] = None
    hidden_states: Optional[Tuple[paddle.Tensor]] = None
    attentions: Optional[Tuple[paddle.Tensor]] = None
    rope_deltas: Optional[paddle.Tensor] = None

### 1.1.6 Qwen2RotaryEmbedding

**Qwen2RotaryEmbedding**

这是一个实现旋转位置编码（Rotary Position Embedding, RoPE）的模块：

__init__:
初始化时，基于 dim（嵌入维度）和最大序列长度，计算 inv_freq（逆频率）。
使用逆频率和位置索引生成预计算的正弦 (cos_cached) 和余弦 (sin_cached) 编码，节约计算时间。

**_set_cos_sin_cache:**
动态更新缓存，确保支持超出预定义长度的序列。

**forward:**
根据输入 x 和序列长度 seq_len 返回对应的正弦和余弦值，以供旋转位置编码使用。

**作用：** RoPE 是一种位置编码方法，允许模型通过对查询和键嵌入进行旋转操作，注入序列中的位置信息，尤其适合 Transformer 架构

In [ ]:
class Qwen2RotaryEmbedding(nn.Layer):
    def __init__(self, dim, max_position_embeddings=2048, base=10000):
        super().__init__()
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        # [dim / 2]
        self.inv_freq = 1.0 / (self.base ** (paddle.cast(paddle.arange(0, self.dim, 2), dtype="float32") / self.dim))
        self._set_cos_sin_cache(seq_len=max_position_embeddings)

    def _set_cos_sin_cache(self, seq_len):
        self.max_seq_len_cached = seq_len
        # [seq_len]
        t = paddle.arange(seq_len, dtype="float32")
        # [seq_len, dim/2]
        freqs = paddle.einsum("i,j->ij", t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        # [seq_len, dim]
        emb = paddle.concat([freqs, freqs], axis=-1)
        # [1, seqlen, 1, dim]
        self.cos_cached = emb.cos()
        self.sin_cached = emb.sin()

    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        if seq_len > self.max_seq_len_cached:
            self._set_cos_sin_cache(seq_len)
        cos = self.cos_cached[:seq_len]
        sin = self.sin_cached[:seq_len]
        return (
            cos.cast(x.dtype) if cos.dtype != x.dtype else cos,
            sin.cast(x.dtype) if sin.dtype != x.dtype else sin,
        )


# 统一视觉处理方式
Qwen2VL采用混合训练方案，结合图像和视频数据，确保图像理解和视频理解的熟练程度。

为了尽可能完整地保存视频信息，Qwen2-VL以每秒两帧的频率对每个视频进行采样。

集成了深度为2的3D卷积来处理视频输入，使模型能够处理3D tubes 而不是2D patches，从而使其能够在不增加序列长度的情况下处理更多的视频帧。

为了保持一致性，每个图像都被视为两个相同的帧。

为了平衡长视频处理的计算需求和整体训练效率，我们动态调整每个视频帧的分辨率，将每个视频的token总数限制在16384。这种训练方法在模型理解长视频的能力和训练效率之间取得了平衡。


                            

# 原生动态分辨率处理
Qwen2 VL的一个关键架构改进是引入了原生动态分辨率支持。与Qwen-VL不同，Qwen2-VL可以处理任意分辨率的图像，将其动态转换为可变数量的视觉标记。

为了支持这一功能，Qwen2-VL修改了ViT，删除了原始的绝对位置嵌入，并引入了**2D RoPE**来捕获图像的二维位置信息。

在推理阶段，不同分辨率的图像被打包成一个序列，打包长度受到控制以限制GPU内存的使用。

此外，为了减少每个图像的视觉标记，在ViT之后使用一个简单的MLP层将相邻的2×2标记压缩成一个标记，并在压缩的视觉标记的开头和结尾放置特殊的**<|vision_start|>**和**<|visition_end|>**标记。

**这一版本的Qwen2-VL并没有采用当下流行的大图切分方式**（比如LLava-Next，InternVL2.5，以及MiniCPM-V），而是直接对图像进行**patch**化，然后直接过image encoder进行特征提取，最后对齐到LLM之前，使用**PatchMerger**层进行视觉token数的压缩与进一步提取特征（两层MLP）。




### 1.1.7 VisionRotaryEmbedding

**VisionRotaryEmbedding**

**功能：** 用于计算视觉数据的旋转位置嵌入（Rotary Position Embedding, RPE）。

**具体功能：**
初始化时根据给定的维度 dim 和比例参数 theta 计算逆频率（inv_freq），用于生成不同维度的旋转嵌入频率。

在 **forward**方法中，基于序列长度 seqlen 和逆频率生成频率矩阵 freqs。

**作用：**
生成与视觉数据位置相关的嵌入信息，用于将位置信息注入到视觉特征中。

In [ ]:
class VisionRotaryEmbedding(nn.Layer):
    def __init__(self, dim: int, theta: float = 10000.0) -> None:
        super().__init__()
        self.inv_freq = 1.0 / theta ** (paddle.arange(start=0, end=dim, step=2, dtype="float32") / dim)

    def forward(self, seqlen: int) -> paddle.Tensor:
        seq = paddle.arange(seqlen).cast(self.inv_freq.dtype)
        freqs = paddle.outer(x=seq, y=self.inv_freq)
        return freqs

### 1.1.8 PatchEmbed

**PatchEmbed**

**功能：** 将输入的图像或视频数据分割成小块（Patch），并通过卷积操作提取特征，生成低维嵌入表示。

**具体功能：**
初始化时定义了一个 3D 卷积层（Conv3D），其卷积核大小由 patch_size（空间维度）和 temporal_patch_size（时间维度）决定，用于提取时空特征。

在 forward 方法中，将输入的视觉数据重新调整为适合 3D 卷积的形状，然后通过卷积层投影到低维特征空间。

针对特定硬件（如 NPU），增加了支持 FP16 或 BF16 数据类型的特定处理逻辑。

**作用：**
将高维的视觉数据（图像或视频帧）映射为低维特征嵌入，作为后续处理的输入。

In [ ]:
class PatchEmbed(nn.Layer):
    def __init__(
        self,
        patch_size: int = 14,
        temporal_patch_size: int = 2,
        in_channels: int = 3,
        embed_dim: int = 1152,
    ) -> None:
        super().__init__()
        self.patch_size = patch_size
        self.temporal_patch_size = temporal_patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim

        kernel_size = [temporal_patch_size, patch_size, patch_size]
        self.proj = nn.Conv3D(in_channels, embed_dim, kernel_size=kernel_size, stride=kernel_size, bias_attr=False)

    def forward(self, hidden_states: paddle.Tensor) -> paddle.Tensor:

        target_dtype = self.proj.weight.dtype
        hidden_states = hidden_states.reshape(
            [-1, self.in_channels, self.temporal_patch_size, self.patch_size, self.patch_size]
        )

        if _IS_NPU:
            # NOTE: In npu device, conv3d only support fp16 or bf16 dtype.
            hidden_states = F.conv3d(
                hidden_states.cast(paddle.bfloat16),
                self.proj.weight.cast(paddle.bfloat16),
                stride=self.proj._stride)
            hidden_states = hidden_states.to(target_dtype).reshape([-1, self.embed_dim])
        else:
            hidden_states = self.proj(hidden_states.to(dtype=target_dtype)).reshape([-1, self.embed_dim])
        return hidden_states

### 1.1.9 PatchMerger

**PatchMerger**

**功能：** 合并视觉特征块（Patches），通过融合邻近特征块的信息，生成新的特征表示。

**具体功能：**
初始化时：
定义了一个 LayerNorm 层（ln_q）进行特征的标准化。

定义了一个简单的全连接层（MLP）用于非线性变换。

**在 forward 方法中：**
对输入特征进行标准化，并调整形状后通过 MLP 进行特征融合。

**作用：**
通过特征标准化和非线性变换，增强特征的表达能力。

In [ ]:
class PatchMerger(nn.Layer):
    def __init__(self, dim: int, context_dim: int, spatial_merge_size: int = 2) -> None:
        super().__init__()
        self.hidden_size = context_dim * (spatial_merge_size**2)
        self.ln_q = nn.LayerNorm(context_dim, epsilon=1e-6)
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.GELU(),
            nn.Linear(self.hidden_size, dim),
        )

    def forward(self, x: paddle.Tensor) -> paddle.Tensor:
        x = self.mlp(self.ln_q(x).reshape([-1, self.hidden_size]))
        return x

### 1.1.10 VisionMlp

**VisionMlp**

**功能：** 一个经典的多层感知机（MLP），用于对视觉特征进行进一步的非线性变换。

**具体功能：**
初始化时：
定义了两层全连接层（fc1 和 fc2）。
使用 hidden_act 指定的激活函数（如 ReLU 或 GELU）进行非线性变换。

**在 forward 方法中：**
将输入依次通过 fc1、激活函数、fc2，生成新的特征表示。

**作用：**
对视觉特征进行非线性处理，提升特征的表达能力，适配后续任务。

In [ ]:
class VisionMlp(nn.Layer):
    def __init__(self, dim: int, hidden_dim: int, hidden_act: str) -> None:
        super().__init__()
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.act = ACT2FN[hidden_act]
        self.fc2 = nn.Linear(hidden_dim, dim)

    def forward(self, x) -> paddle.Tensor:
        return self.fc2(self.act(self.fc1(x)))

### 1.1.11 VisionAttention

**VisionAttention**

**初始化方法**

**dim:** 输入特征的维度。

**num_heads:** 注意力头的数量。

计算 head_dim，即每个头分配的维度大小（dim // num_heads）。


**前向传播方法：**

**输入：**
**hidden_states:** 输入的特征张量，形状为 (seq_length, dim)。

**cu_seqlens:** 表示累积序列长度，用于区分批次的序列范围。

**rotary_pos_emb:** 旋转位置嵌入，用于增强位置编码的能力。

**过程：**

使用 qkv 全连接层将输入分解为查询 (Q)、键 (K)、值 (V)。

使用 apply_rotary_pos_emb_vision 应用旋转位置嵌入。

构造注意力掩码，用于屏蔽不相关的位置。

计算注意力分数 (attn_weights) 并应用 softmax。

根据权重聚合值 (V) 得到注意力输出。

使用 proj 层投影输出。

**输出：**
注意力模块的输出特征。

In [ ]:
class VisionAttention(nn.Layer):
    def __init__(self, dim: int, num_heads: int = 16) -> None:
        super().__init__()
        self.num_heads = num_heads
        self.qkv = nn.Linear(dim, dim * 3, bias_attr=True)
        self.proj = nn.Linear(dim, dim)
        self.head_dim = dim // num_heads  # must added

    def forward(
        self, hidden_states: paddle.Tensor, cu_seqlens: paddle.Tensor, rotary_pos_emb: paddle.Tensor = None
    ) -> paddle.Tensor:
        seq_length = hidden_states.shape[0]
        q, k, v = (
            self.qkv(hidden_states).reshape([seq_length, 3, self.num_heads, -1]).transpose([1, 0, 2, 3]).unbind(0)
        )
        q = apply_rotary_pos_emb_vision(q.unsqueeze(0), rotary_pos_emb).squeeze(0)
        k = apply_rotary_pos_emb_vision(k.unsqueeze(0), rotary_pos_emb).squeeze(0)

        attention_mask = paddle.zeros([1, seq_length, seq_length], dtype="bool")
        for i in range(1, len(cu_seqlens)):
            attention_mask[..., cu_seqlens[i - 1] : cu_seqlens[i], cu_seqlens[i - 1] : cu_seqlens[i]] = True

        zero = paddle.zeros(attention_mask.shape, dtype=hidden_states.dtype)
        neg_inf = paddle.full_like(attention_mask, paddle.finfo(hidden_states.dtype).min, dtype=hidden_states.dtype)
        attention_mask = paddle.where(attention_mask, zero, neg_inf)

        q = q.transpose([1, 0, 2])
        k = k.transpose([1, 0, 2])
        v = v.transpose([1, 0, 2])
        attn_weights = paddle.matmul(q, k.transpose([0, 2, 1])) / math.sqrt(self.head_dim)
        attn_weights = attn_weights + attention_mask
        attn_weights = nn.functional.softmax(attn_weights, axis=-1, dtype="float32")
        attn_output = paddle.matmul(attn_weights, v)
        attn_output = attn_output.transpose([1, 0, 2])
        attn_output = attn_output.reshape([seq_length, -1])
        attn_output = self.proj(attn_output)
        return attn_output


### 1.1.12 VisionFlashAttention2

**VisionFlashAttention2**

这是一个优化版本的注意力实现，利用了**Flash Attention**的快速计算特性。

**与 VisionAttention 的区别：**

利用 flash_attn_varlen_func 优化了注意力分数的计算。
直接处理长序列的注意力计算，提升速度和效率。
所有计算在 bfloat16 类型下进行，最后转回 float32。

**新增参数：**
softmax_scale: 手动加上的缩放因子（head_dim**-0.5）。

**改进：**
支持变长序列的高效计算。
更适合处理大规模视觉任务。

In [ ]:
class VisionFlashAttention2(nn.Layer):
    def __init__(self, dim: int, num_heads: int = 16) -> None:
        super().__init__()
        self.num_heads = num_heads
        self.qkv = nn.Linear(dim, dim * 3, bias_attr=True)
        self.proj = nn.Linear(dim, dim)
        self.head_dim = dim // num_heads  # must added

    def forward(
        self, hidden_states: paddle.Tensor, cu_seqlens: paddle.Tensor, rotary_pos_emb: paddle.Tensor = None
    ) -> paddle.Tensor:
        seq_length = tuple(hidden_states.shape)[0]
        qkv = self.qkv(hidden_states).reshape([seq_length, 3, self.num_heads, -1]).transpose(perm=[1, 0, 2, 3])
        q, k, v = qkv.unbind(axis=0)
        q = apply_rotary_pos_emb_vision(q.unsqueeze(axis=0), rotary_pos_emb).squeeze(axis=0)
        k = apply_rotary_pos_emb_vision(k.unsqueeze(axis=0), rotary_pos_emb).squeeze(axis=0)
        max_seqlen = (cu_seqlens[1:] - cu_seqlens[:-1]).max().item()

        softmax_scale = self.head_dim**-0.5  # TODO: 需要手动加上
        attn_output = (
            flash_attn_varlen_func(  # flash_attn_unpadded
                q.astype("bfloat16"),  # 不支持float32
                k.astype("bfloat16"),
                v.astype("bfloat16"),
                cu_seqlens,
                cu_seqlens,
                max_seqlen,
                max_seqlen,
                scale=softmax_scale,  # TODO: 需要手动加上
            )[0]
            .squeeze(0)
            .reshape([seq_length, -1])
        )
        attn_output = attn_output.astype(paddle.float32)
        attn_output = self.proj(attn_output)
        return attn_output


### 1.1.12 Qwen2VLVisionBlock

**Qwen2VLVisionBlock**

视觉模型的基本块，包含注意力和前馈网络。

**结构：**

包含两个子模块：
注意力模块 (self.attn)。
前馈网络 (self.mlp)。

两个子模块之间通过残差连接和层归一化进行信息传递。

**前向传播方法：**

对输入依次应用注意力模块和前馈网络。
每个模块都包含残差连接，防止梯度消失。

In [ ]:
class Qwen2VLVisionBlock(nn.Layer):
    def __init__(self, config, attn_implementation: str = "sdpa") -> None:
        super().__init__()
        self.norm1 = nn.LayerNorm(config.embed_dim, epsilon=1e-6)
        self.norm2 = nn.LayerNorm(config.embed_dim, epsilon=1e-6)
        mlp_hidden_dim = int(config.embed_dim * config.mlp_ratio)

        self.attn = create_attention_module(config, "vision")
        self.mlp = VisionMlp(dim=config.embed_dim, hidden_dim=mlp_hidden_dim, hidden_act=config.hidden_act)

    def forward(self, hidden_states, cu_seqlens, rotary_pos_emb) -> paddle.Tensor:
        hidden_states = hidden_states + self.attn(
            self.norm1(hidden_states), cu_seqlens=cu_seqlens, rotary_pos_emb=rotary_pos_emb
        )
        hidden_states = hidden_states + self.mlp(self.norm2(hidden_states))
        return hidden_states

### 1.1.13 _prepare_4d_causal_attention_mask_with_cache_position

**_prepare_4d_causal_attention_mask_with_cache_position**

生成一个4D的因果掩码，用于控制注意力机制中的信息流。

**功能**

将二维的注意力掩码扩展为四维。

支持生成过程中静态缓存的处理。

**主要步骤**

如果输入已经是四维，直接返回。
否则，创建一个上三角矩阵，确保注意力只能流向过去或当前时间步。
根据 cache_position 和输入掩码动态调整矩阵。
扩展为 [batch_size, 1, query_length, key_value_length] 的形状。

**关键参数**

**min_dtype:** 表示数值的最小值（负无穷），用于屏蔽注意力。

**cache_position:** 用于跟踪当前生成的位置。

In [ ]:
def _prepare_4d_causal_attention_mask_with_cache_position(
    attention_mask: paddle.Tensor,
    sequence_length: int,
    target_length: int,
    dtype: paddle.dtype,
    min_dtype: float,
    cache_position: paddle.Tensor,
    batch_size: int,
):
    """
    Creates a causal 4D mask of shape `(batch_size, 1, query_length, key_value_length)` from a 2D mask of shape
    `(batch_size, key_value_length)`, or if the input `attention_mask` is already 4D, do nothing.

    Args:
        attention_mask (`paddle.Tensor`):
            A 2D attention mask of shape `(batch_size, key_value_length)` or a 4D attention mask of shape `(batch_size, 1, query_length, key_value_length)`.
        sequence_length (`int`):
            The sequence length being processed.
        target_length (`int`):
            The target length: when generating with static cache, the mask should be as long as the static cache, to account for the 0 padding, the part of the cache that is not filled yet.
        dtype (`paddle.dtype`):
            The dtype to use for the 4D attention mask.
        min_dtype (`float`):
            The minimum value representable with the dtype `dtype`.
        cache_position (`paddle.Tensor`):
            Indices depicting the position of the input sequence tokens in the sequence.
        batch_size (`paddle.Tensor`):
            Batch size.
    """
    if attention_mask is not None and attention_mask.dim() == 4:
        # In this case we assume that the mask comes already in inverted form and requires no inversion or slicing.
        causal_mask = attention_mask
    else:
        causal_mask = paddle.full([sequence_length, target_length], fill_value=min_dtype, dtype=dtype)
        if sequence_length != 1:
            causal_mask = paddle.triu(x=causal_mask, diagonal=1)
        causal_mask *= paddle.arange(target_length) > cache_position.reshape([-1, 1])
        causal_mask = causal_mask[None, None, :, :].expand(shape=[batch_size, 1, -1, -1])
        if attention_mask is not None:
            causal_mask = causal_mask.clone()
            mask_length = tuple(attention_mask.shape)[-1]
            padding_mask = causal_mask[:, :, :, :mask_length] + attention_mask[:, None, None, :]
            padding_mask = padding_mask == 0
            causal_mask[:, :, :, :mask_length] = causal_mask[:, :, :, :mask_length].masked_fill(
                mask=padding_mask, value=min_dtype
            )

    return causal_mask

### 1.1.14 Qwen2VLVisionBlock

**Qwen2VLVisionBlock**

视觉模型的基本块，包含注意力和前馈网络。

**结构：**

包含两个子模块：
注意力模块 (self.attn)。
前馈网络 (self.mlp)。

两个子模块之间通过残差连接和层归一化进行信息传递。

**前向传播方法：**

对输入依次应用注意力模块和前馈网络。
每个模块都包含残差连接，防止梯度消失。

In [ ]:
class Qwen2VLVisionBlock(nn.Layer):
    def __init__(self, config, attn_implementation: str = "sdpa") -> None:
        super().__init__()
        self.norm1 = nn.LayerNorm(config.embed_dim, epsilon=1e-6)
        self.norm2 = nn.LayerNorm(config.embed_dim, epsilon=1e-6)
        mlp_hidden_dim = int(config.embed_dim * config.mlp_ratio)

        self.attn = create_attention_module(config, "vision")
        self.mlp = VisionMlp(dim=config.embed_dim, hidden_dim=mlp_hidden_dim, hidden_act=config.hidden_act)

    def forward(self, hidden_states, cu_seqlens, rotary_pos_emb) -> paddle.Tensor:
        hidden_states = hidden_states + self.attn(
            self.norm1(hidden_states), cu_seqlens=cu_seqlens, rotary_pos_emb=rotary_pos_emb
        )
        hidden_states = hidden_states + self.mlp(self.norm2(hidden_states))
        return hidden_states


# Copied from transformers.models.llama.modeling_llama._prepare_4d_causal_attention_mask_with_cache_position
def _prepare_4d_causal_attention_mask_with_cache_position(
    attention_mask: paddle.Tensor,
    sequence_length: int,
    target_length: int,
    dtype: paddle.dtype,
    min_dtype: float,
    cache_position: paddle.Tensor,
    batch_size: int,
):
    """
    Creates a causal 4D mask of shape `(batch_size, 1, query_length, key_value_length)` from a 2D mask of shape
    `(batch_size, key_value_length)`, or if the input `attention_mask` is already 4D, do nothing.

    Args:
        attention_mask (`paddle.Tensor`):
            A 2D attention mask of shape `(batch_size, key_value_length)` or a 4D attention mask of shape `(batch_size, 1, query_length, key_value_length)`.
        sequence_length (`int`):
            The sequence length being processed.
        target_length (`int`):
            The target length: when generating with static cache, the mask should be as long as the static cache, to account for the 0 padding, the part of the cache that is not filled yet.
        dtype (`paddle.dtype`):
            The dtype to use for the 4D attention mask.
        min_dtype (`float`):
            The minimum value representable with the dtype `dtype`.
        cache_position (`paddle.Tensor`):
            Indices depicting the position of the input sequence tokens in the sequence.
        batch_size (`paddle.Tensor`):
            Batch size.
    """
    if attention_mask is not None and attention_mask.dim() == 4:
        # In this case we assume that the mask comes already in inverted form and requires no inversion or slicing.
        causal_mask = attention_mask
    else:
        causal_mask = paddle.full([sequence_length, target_length], fill_value=min_dtype, dtype=dtype)
        if sequence_length != 1:
            causal_mask = paddle.triu(x=causal_mask, diagonal=1)
        causal_mask *= paddle.arange(target_length) > cache_position.reshape([-1, 1])
        causal_mask = causal_mask[None, None, :, :].expand(shape=[batch_size, 1, -1, -1])
        if attention_mask is not None:
            causal_mask = causal_mask.clone()
            mask_length = tuple(attention_mask.shape)[-1]
            padding_mask = causal_mask[:, :, :, :mask_length] + attention_mask[:, None, None, :]
            padding_mask = padding_mask == 0
            causal_mask[:, :, :, :mask_length] = causal_mask[:, :, :, :mask_length].masked_fill(
                mask=padding_mask, value=min_dtype
            )

    return causal_mask

### 1.1.15 Qwen2RMSNorm

**Qwen2RMSNorm**

效于 T5LayerNorm，通过均方根归一化（RMSNorm）实现对输入的归一化：

**实现思路：**

对输入张量按最后一个维度计算均值平方差（方差）。

通过平方差开平方得到归一化因子，避免数值不稳定。

使用训练的权重参数调整归一化后的值。

In [ ]:
class Qwen2RMSNorm(nn.Layer):
    def __init__(self, hidden_size, eps=1e-6):
        """
        Qwen2RMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = paddle.create_parameter(
            shape=[hidden_size],
            dtype=paddle.get_default_dtype(),
            default_initializer=nn.initializer.Constant(1.0),
        )
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        if paddle.in_dynamic_mode():
            with paddle.amp.auto_cast(False):
                variance = hidden_states.astype("float32").pow(2).mean(-1, keepdim=True)
                hidden_states = paddle.rsqrt(variance + self.variance_epsilon) * hidden_states
        else:
            variance = hidden_states.astype("float32").pow(2).mean(-1, keepdim=True)
            hidden_states = paddle.rsqrt(variance + self.variance_epsilon) * hidden_states

        if self.weight.dtype in [paddle.float16, paddle.bfloat16]:
            hidden_states = paddle.cast(hidden_states, self.weight.dtype)
        return hidden_states * self.weight

### 1.1.16 Qwen2MLP

**Qwen2MLP**

实现基于门控激活的多层感知机（MLP）

**结构：**
输入通过两个线性层分别计算门控值（gate_proj）和变换值（up_proj）。

激活函数（例如GELU或ReLU）应用于门控值。

最终通过另一个线性层（down_proj）恢复到原始维度。

**核心逻辑：**

将非线性激活和线性投影相结合，增强了模型的表达能力。

In [ ]:
class Qwen2MLP(nn.Layer):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias_attr=False)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias_attr=False)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias_attr=False)
        self.act_fn = ACT2FN[config.hidden_act]

    def forward(self, hidden_state):
        return self.down_proj(self.act_fn(self.gate_proj(hidden_state)) * self.up_proj(hidden_state))

### 1.1.17 Qwen2VLAttention

**Qwen2VLAttention**

标准多头注意力机制，结合了旋转位置嵌入（RoPE）以增强长序列处理能力

**主要模块：**
投影层（q_proj，k_proj，v_proj）将输入特征投影为查询、键和值。

使用旋转嵌入调整键和值以融入位置信息。

点积注意力计算 softmax(QK^T / sqrt(d))。

**支持功能：**

缓存（past_key_value）：对长序列应用时复用计算结果。

动态掩码（attention_mask）：处理因填充导致的无效区域。

In [ ]:
class Qwen2VLAttention(nn.Layer):
    """
    Multi-headed attention from 'Attention Is All You Need' paper. Modified to use sliding window attention: Longformer
    and "Generating Long Sequences with Sparse Transformers".
    """

    def __init__(self, config: Qwen2VLConfig, layer_idx: Optional[int] = None):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        if layer_idx is None:
            logger.warning_once(
                f"Instantiating {self.__class__.__name__} without passing `layer_idx` is not recommended and will "
                "to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` "
                "when creating this class."
            )

        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.rope_theta = config.rope_theta
        self.is_causal = True
        self.attention_dropout = config.attention_dropout
        self.rope_scaling = config.rope_scaling

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )
        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias_attr=True)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias_attr=True)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias_attr=True)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias_attr=False)

        self.rotary_emb = Qwen2RotaryEmbedding(
            self.head_dim,
            max_position_embeddings=self.max_position_embeddings,
            base=self.rope_theta,
        )

    def forward(
        self,
        hidden_states: paddle.Tensor,
        attention_mask: Optional[paddle.Tensor] = None,
        position_ids: Optional[paddle.Tensor] = None,
        past_key_value: Optional[Tuple[paddle.Tensor]] = None,  # Cache
        output_attentions: bool = False,
        use_cache: bool = False,  # default true
        cache_position: Optional[paddle.Tensor] = None,
    ) -> Tuple[paddle.Tensor, Optional[paddle.Tensor], Optional[Tuple[paddle.Tensor]]]:
        bsz, q_len, _ = hidden_states.shape

        try:
            query_states = self.q_proj(hidden_states)
            key_states = self.k_proj(hidden_states)
            value_states = self.v_proj(hidden_states)
        except:
            hidden_states = hidden_states.astype("bfloat16")
            query_states = self.q_proj(hidden_states)
            key_states = self.k_proj(hidden_states)
            value_states = self.v_proj(hidden_states)

        new_perm = [0, 2, 1, 3]
        query_states = query_states.reshape([bsz, q_len, self.num_heads, self.head_dim]).transpose(new_perm)
        key_states = key_states.reshape([bsz, q_len, self.num_key_value_heads, self.head_dim]).transpose(new_perm)
        value_states = value_states.reshape([bsz, q_len, self.num_key_value_heads, self.head_dim]).transpose(new_perm)

        kv_seq_len = key_states.shape[-2]  # q_len ######## [bs, num_head, seq_len, head_dim]      # qwen2是 [-3]
        if past_key_value is not None:
            kv_seq_len += cache_position[0] + 1
            # kv_seq_len += past_key_value[0].shape[-2] # qwen2是 [-3]

        cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
        query_states, key_states = apply_multimodal_rotary_pos_emb(
            query_states, key_states, cos, sin, position_ids, self.rope_scaling["mrope_section"]
        )

        # [bs, num_head, seq_len, head_dim]
        if past_key_value is not None:
            # cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            # key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)
            key_states = paddle.concat([past_key_value[0], key_states], axis=2)  # qwen2是 axis=1, qwen2_vl是 axis=2
            value_states = paddle.concat([past_key_value[1], value_states], axis=2)  # qwen2是 axis=1
        past_key_value = (key_states, value_states) if use_cache else None

        # repeat k/v heads if n_kv_heads < n_heads
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)

        attn_weights = paddle.matmul(query_states, key_states.transpose([0, 1, 3, 2])) / math.sqrt(self.head_dim)

        if attention_mask is not None:  # no matter the length, we just slice it
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
            attn_weights = attn_weights + causal_mask

        # upcast attention to fp32
        attn_weights = nn.functional.softmax(x=attn_weights, axis=-1, dtype="float32")
        attn_weights = nn.functional.dropout(x=attn_weights, p=self.attention_dropout, training=self.training)
        attn_output = paddle.matmul(attn_weights.cast("bfloat16"), value_states.cast("bfloat16"))  # TODO: hard code

        if attn_output.shape != [bsz, self.num_heads, q_len, self.head_dim]:
            raise ValueError(
                f"`attn_output` should be of size {(bsz, q_len, self.num_heads, self.head_dim)}, but is"
                f" {attn_output.shape}"
            )

        attn_output = attn_output.transpose([0, 2, 1, 3])
        attn_output = attn_output.reshape([bsz, q_len, -1])
        attn_output = self.o_proj(attn_output)
        if not output_attentions:
            attn_weights = None
        return attn_output, attn_weights, past_key_value

### 1.1.18 Qwen2VLFlashAttention2

**Qwen2VLFlashAttention2**

基于 **Flash Attention** 的实现，适用于更高效的序列建模

In [ ]:
class Qwen2VLFlashAttention2(Qwen2VLAttention):
    """
    Qwen2VL flash attention module, following Qwen2VL attention module. This module inherits from `Qwen2VLAttention`
    as the weights of the module stays untouched. The only required change would be on the forward pass
    where it needs to correctly call the public API of flash attention and deal with padding tokens
    in case the input contains any of them. Additionally, for sliding window attention, we apply SWA only to the bottom
    config.max_window_layers layers.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(
        self,
        hidden_states: paddle.Tensor,
        attention_mask: Optional[paddle.Tensor] = None,
        position_ids: Optional[paddle.Tensor] = None,
        past_key_value: Optional[Tuple[paddle.Tensor]] = None,  # Cache
        output_attentions: bool = False,
        use_cache: bool = False,  # default true
        cache_position: Optional[paddle.Tensor] = None,
    ) -> Tuple[paddle.Tensor, Optional[paddle.Tensor], Optional[Tuple[paddle.Tensor]]]:
        bsz, q_len, _ = tuple(hidden_states.shape)

        try:
            query_states = self.q_proj(hidden_states)
            key_states = self.k_proj(hidden_states)
            value_states = self.v_proj(hidden_states)
        except:
            hidden_states = hidden_states.astype("bfloat16")
            query_states = self.q_proj(hidden_states)
            key_states = self.k_proj(hidden_states)
            value_states = self.v_proj(hidden_states)

        new_perm = [0, 2, 1, 3]
        # [1, 3599, 1536] [bsz, q_len, self.num_heads * self.head_dim]
        query_states = query_states.reshape([bsz, q_len, self.num_heads, self.head_dim]).transpose(new_perm)
        key_states = key_states.reshape([bsz, q_len, self.num_key_value_heads, self.head_dim]).transpose(new_perm)
        value_states = value_states.reshape([bsz, q_len, self.num_key_value_heads, self.head_dim]).transpose(new_perm)

        kv_seq_len = key_states.shape[-2]  # q_len ######## [bs, num_head, seq_len, head_dim]      # qwen2是 [-3]
        if past_key_value is not None:
            kv_seq_len += cache_position[0] + 1

        # Because the input can be padded, the absolute sequence length depends on the max position id.
        cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
        query_states, key_states = apply_multimodal_rotary_pos_emb(
            query_states, key_states, cos, sin, position_ids, self.rope_scaling["mrope_section"]
        )

        if past_key_value is not None:
            # cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            # key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)
            key_states = paddle.concat([past_key_value[0], key_states], axis=2)  # qwen2是 axis=1, qwen2_vl是 axis=2
            value_states = paddle.concat([past_key_value[1], value_states], axis=2)  # qwen2是 axis=1
        past_key_value = (key_states, value_states) if use_cache else None

        # repeat k/v heads if n_kv_heads < n_heads
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)

        # Reashape to the expected shape for Flash Attention
        # [1, 3599, 12, 128]
        query_states = query_states.transpose(perm=[0, 2, 1, 3])
        key_states = key_states.transpose(perm=[0, 2, 1, 3])
        value_states = value_states.transpose(perm=[0, 2, 1, 3])

        attn_output = self._flash_attention_forward(
            query_states,
            key_states,
            value_states,
            None,
            q_len
            # dropout=0.0 if not self.training else self.attention_dropout,
            # causal=self.is_causal,
        )

        attn_output = attn_output.reshape([bsz, q_len, self.hidden_size])
        attn_output = self.o_proj(attn_output)
        if not output_attentions:
            attn_weights = None
        return attn_output, attn_weights, past_key_value

    def _flash_attention_forward(
        self, query_states, key_states, value_states, attention_mask, query_length, dropout=0.0, softmax_scale=None
    ):
        """
        Calls the forward method of Flash Attention - if the input hidden states contain at least one padding token
        first unpad the input, then computes the attention scores and pad the final attention scores.

        Args:
            query_states (`paddle.Tensor`):
                Input query states to be passed to Flash Attention API
            key_states (`paddle.Tensor`):
                Input key states to be passed to Flash Attention API
            value_states (`paddle.Tensor`):
                Input value states to be passed to Flash Attention API
            attention_mask (`paddle.Tensor`):
                The padding mask - corresponds to a tensor of size `(batch_size, seq_len)` where 0 stands for the
                position of padding tokens and 1 for the position of non-padding tokens.
            dropout (`int`, *optional*):
                Attention dropout
            softmax_scale (`float`, *optional*):
                The scaling of QK^T before applying softmax. Default to 1 / sqrt(head_dim)
        """
        # Contains at least one padding token in the sequence
        causal = self.is_causal and query_length != 1

        head_dim = query_states.shape[-1]
        softmax_scale = head_dim**-0.5  # TODO: 需要手动加上

        if attention_mask is not None:
            batch_size = query_states.shape[0]  # [2, 3383, 16, 128]

            query_states, key_states, value_states, indices_q, cu_seq_lens, max_seq_lens = self._unpad_input(
                query_states, key_states, value_states, attention_mask, query_length
            )

            cu_seqlens_q, cu_seqlens_k = cu_seq_lens
            max_seqlen_in_batch_q, max_seqlen_in_batch_k = max_seq_lens

            attn_output_unpad = flash_attn_varlen_func(  # TODO: flash_attn_unpadded
                query_states,  # [5998, 16, 128]
                key_states,  # [5998, 8, 128]
                value_states,  # [5998, 8, 128]
                cu_seqlens_q=cu_seqlens_q,
                cu_seqlens_k=cu_seqlens_k,
                max_seqlen_q=max_seqlen_in_batch_q,
                max_seqlen_k=max_seqlen_in_batch_k,
                scale=softmax_scale,  # not softmax_scale=
                dropout=dropout,
                causal=causal,
            )[0]

            attn_output = pad_input(attn_output_unpad, indices_q, batch_size, query_length)
        else:
            attn_output = flash_attn_func(
                query_states,
                key_states,
                value_states,
                dropout,
                causal=causal,  # no softmax_scale=
            )[0]

        return attn_output

    def _unpad_input(self, query_layer, key_layer, value_layer, attention_mask, query_length):
        indices_k, cu_seqlens_k, max_seqlen_in_batch_k = _get_unpad_data(attention_mask)
        batch_size, kv_seq_len, num_key_value_heads, head_dim = key_layer.shape

        key_layer = index_first_axis(
            key_layer.reshape([batch_size * kv_seq_len, num_key_value_heads, head_dim]), indices_k
        )
        value_layer = index_first_axis(
            value_layer.reshape([batch_size * kv_seq_len, num_key_value_heads, head_dim]), indices_k
        )

        if query_length == kv_seq_len:
            query_layer = index_first_axis(
                query_layer.reshape([batch_size * kv_seq_len, self.num_heads, head_dim]), indices_k
            )
            cu_seqlens_q = cu_seqlens_k
            max_seqlen_in_batch_q = max_seqlen_in_batch_k
            indices_q = indices_k
        elif query_length == 1:
            max_seqlen_in_batch_q = 1
            cu_seqlens_q = paddle.arange(
                batch_size + 1, dtype=paddle.int32
            )  # There is a memcpy here, that is very bad.
            indices_q = cu_seqlens_q[:-1]
            query_layer = query_layer.squeeze(1)
        else:
            # The -q_len: slice assumes left padding.
            attention_mask = attention_mask[:, -query_length:]
            query_layer, indices_q, cu_seqlens_q, max_seqlen_in_batch_q = unpad_input(query_layer, attention_mask)

        return (
            query_layer,
            key_layer,
            value_layer,
            indices_q.to(paddle.int64),
            (cu_seqlens_q, cu_seqlens_k),
            (max_seqlen_in_batch_q, max_seqlen_in_batch_k),
        )

### 1.1.19 Qwen2VLDecoderLayer

**Qwen2VLDecoderLayer**

**功能**

这是一个Transformer解码器的实现，包含两个主要部分：

**自注意力模块（Self Attention）：**用于捕获输入序列中各位置之间的全局依赖关系。

**前馈神经网络模块（Feed-Forward Network, FFN）：**用于非线性变换，增强表示能力。


**self.self_attn:** 自注意力机制模块。

**self.mlp:** 两层线性变换和激活函数构成的前馈神经网络。

**LayerNorm:** 使用RMS归一化方法对输入数据进行标准化。

**残差连接:** 将每个子层的输入加到输出中，以缓解梯度消失问题并促进信息流动。

**前向传播（forward）流程**

1.对输入状态进行标准化（self.input_layernorm）。

2.通过自注意力模块，计算更新后的隐藏状态。

3.对自注意力输出进行标准化，再通过前馈网络进一步处理。

4.将最终的隐藏状态与残差相加，得到模块输出。

5.返回隐藏状态以及（可选的）注意力权重和缓存。

In [ ]:
class Qwen2VLDecoderLayer(nn.Layer):
    def __init__(self, config: Qwen2VLConfig, layer_idx: int):
        super().__init__()
        self.hidden_size = config.hidden_size

        # use_sliding_window false
        if config.use_sliding_window and config.attn_implementation != "flash_attention_2":
            logger.warning_once(
                f"Sliding Window Attention is enabled but not implemented for `{config.attn_implementation}`; "
                "unexpected results may be encountered."
            )
        self.self_attn = create_attention_module(config, "qwen2vl", layer_idx=layer_idx)
        self.mlp = Qwen2MLP(config)
        self.input_layernorm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(
        self,
        hidden_states: paddle.Tensor,
        attention_mask: Optional[paddle.Tensor] = None,
        position_ids: Optional[paddle.Tensor] = None,
        past_key_value: Optional[Tuple[paddle.Tensor]] = None,
        output_attentions: Optional[bool] = False,
        use_cache: Optional[bool] = False,
        cache_position: Optional[paddle.Tensor] = None,
        **kwargs,
    ):
        """
        Args:
            hidden_states (`paddle.Tensor`): input to the layer of shape `(batch, seq_len, embed_dim)`
            attention_mask (`paddle.Tensor`, *optional*): attention mask of size
                `(batch, sequence_length)` where padding elements are indicated by 0.
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
            use_cache (`bool`, *optional*):
                If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                (see `past_key_values`).
            past_key_value (`Tuple(paddle.Tensor)`, *optional*): cached past key and value projection states
            cache_position (`paddle.Tensor` of shape `(sequence_length)`, *optional*):
                Indices depicting the position of the input sequence tokens in the sequence.
            kwargs (`dict`, *optional*):
                Arbitrary kwargs to be ignored, used for FSDP and other methods that injects code
                into the model
        """

        residual = hidden_states

        hidden_states = self.input_layernorm(hidden_states)

        # Self Attention
        hidden_states, self_attn_weights, present_key_value = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_value=past_key_value,
            output_attentions=output_attentions,
            use_cache=use_cache,
            cache_position=cache_position,
        )
        hidden_states = residual + hidden_states

        # Fully Connected
        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states

        outputs = (hidden_states,)

        if output_attentions:
            outputs += (self_attn_weights,)

        if use_cache:
            outputs += (present_key_value,)

        return outputs

### 1.1.20 Qwen2VisionTransformerPretrainedModel

**Qwen2VisionTransformerPretrainedModel**

**功能**
实现了一个视觉Transformer，用于处理图像数据并提取视觉特征。这部分与Transformer编码器类似，但适配了视觉任务需求。

**组成**

**Patch Embedding (self.patch_embed):** 将输入图像切分为小块（patches）并映射到高维嵌入空间。

**Rotary Positional Embedding (self.rotary_pos_emb):** 提供旋转式的位置编码，用于捕捉输入序列的位置信息。

**Blocks (self.blocks):** 多层堆叠的视觉Transformer块，完成对图像特征的逐步提取和处理。

**Merger (self.merger):** 合并多尺度的特征信息。

**前向传播（forward）流程**

**嵌入生成:** 使用PatchEmbed将输入图像转换为高维特征表示。

**位置编码:** 计算输入序列的旋转式位置嵌入。

**逐层处理:** 依次通过视觉Transformer块，更新隐藏状态。

**特征合并:** 使用PatchMerger模块将提取的多尺度视觉特征整合成最终表示。


In [ ]:
class Qwen2VisionTransformerPretrainedModel(Qwen2VLPreTrainedModel):
    config_class = Qwen2VLVisionConfig
    _no_split_modules = ["Qwen2VLVisionBlock"]

    def __init__(self, config) -> None:
        super().__init__(config)
        self.spatial_merge_size = config.spatial_merge_size

        self.patch_embed = PatchEmbed(
            patch_size=config.patch_size,
            temporal_patch_size=config.temporal_patch_size,
            in_channels=config.in_channels,
            embed_dim=config.embed_dim,
        )

        head_dim = config.embed_dim // config.num_heads
        self.rotary_pos_emb = VisionRotaryEmbedding(head_dim // 2)

        self.blocks = nn.LayerList([Qwen2VLVisionBlock(config) for _ in range(config.depth)])
        self.merger = PatchMerger(dim=config.hidden_size, context_dim=config.embed_dim)

    def get_dtype(self) -> paddle.dtype:
        return self.blocks[0].mlp.fc2.weight.dtype

    def rot_pos_emb(self, grid_thw):
        pos_ids = []
        for t, h, w in grid_thw:
            hpos_ids = paddle.arange(h).unsqueeze(1).expand([-1, w])
            hpos_ids = hpos_ids.reshape(
                [
                    h // self.spatial_merge_size,
                    self.spatial_merge_size,
                    w // self.spatial_merge_size,
                    self.spatial_merge_size,
                ]
            )
            hpos_ids = hpos_ids.transpose(perm=[0, 2, 1, 3])
            hpos_ids = hpos_ids.flatten()

            wpos_ids = paddle.arange(w).unsqueeze(0).expand([h, -1])
            wpos_ids = wpos_ids.reshape(
                [
                    h // self.spatial_merge_size,
                    self.spatial_merge_size,
                    w // self.spatial_merge_size,
                    self.spatial_merge_size,
                ]
            )
            wpos_ids = wpos_ids.transpose([0, 2, 1, 3])
            wpos_ids = wpos_ids.flatten()
            pos_ids.append(paddle.stack(x=[hpos_ids, wpos_ids], axis=-1).tile(repeat_times=[t, 1]))
        pos_ids = paddle.concat(x=pos_ids, axis=0)
        max_grid_size = grid_thw[:, 1:].max()
        rotary_pos_emb_full = self.rotary_pos_emb(max_grid_size)
        rotary_pos_emb = rotary_pos_emb_full[pos_ids].flatten(start_axis=1)
        return rotary_pos_emb

    def forward(self, hidden_states: paddle.Tensor, grid_thw: paddle.Tensor) -> paddle.Tensor:

        hidden_states = self.patch_embed(hidden_states)
        rotary_pos_emb = self.rot_pos_emb(grid_thw)

        cu_seqlens = paddle.repeat_interleave(grid_thw[:, 1] * grid_thw[:, 2], grid_thw[:, 0]).cumsum(
            axis=0, dtype="int32"
        )
        cu_seqlens = F.pad(cu_seqlens, (1, 0), value=0)

        for idx, blk in enumerate(self.blocks):
            hidden_states = blk(hidden_states, cu_seqlens=cu_seqlens, rotary_pos_emb=rotary_pos_emb)

        return self.merger(hidden_states)


### 1.1.21 Qwen2VLModel

**Qwen2VLModel**

该类是一个自定义的解码器模型，主要用于处理序列生成任务，如语言模型或序列到序列模型。

关键属性与方法

__init__ 构造方法:

**初始化模型，**包括词嵌入层（self.embed_tokens）、多层解码器层（self.layers）、归一化层（self.norm）以及其他配置参数。

**padding_idx:** 用于处理序列中填充位置（如 PAD token）的索引。

**gradient_checkpointing:** 控制是否启用梯度检查点技术，以减少显存占用。

**get_input_embeddings 和 set_input_embeddings:**获取或设置输入的嵌入层，用于与其他模块共享词嵌入权重。

**_prepare_decoder_attention_mask:**
准备解码器使用的注意力掩码，支持两种掩码

**扩展的输入注意力掩码:** 用于屏蔽填充位置。

**因果掩码:** 确保解码器在生成序列时只能访问过去的信息（自回归特性）。


**forward 方法:**

核心计算逻辑，完成模型的前向传播。主要步骤包括：

**输入处理:** 接受 input_ids 或嵌入表示 inputs_embeds。

**嵌入生成:** 将输入序列映射为嵌入表示。

**注意力掩码计算:** 根据输入生成因果掩码和填充掩码。

**位置编码:** 通过 position_ids 提供位置信息。

**解码层堆叠:** 逐层处理隐藏状态，生成解码器输出。

**返回结果:** 支持多种返回格式，包括隐层状态、注意力权重和缓存。


In [ ]:
class Qwen2VLModel(Qwen2VLPreTrainedModel):
    def __init__(self, config: Qwen2VLConfig):
        super().__init__(config)
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
        self.layers = nn.LayerList(
            [Qwen2VLDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )
        self.norm = Qwen2RMSNorm(config.hidden_size, eps=config.rms_norm_eps)

        self.gradient_checkpointing = False

    def get_input_embeddings(self):
        return self.embed_tokens

    def set_input_embeddings(self, value):
        self.embed_tokens = value

    @staticmethod
    def _prepare_decoder_attention_mask(attention_mask, input_shape, past_key_values_length, dtype):
        if attention_mask is not None:
            # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
            if len(attention_mask.shape) == 2:
                expanded_attn_mask = _expand_2d_mask(attention_mask, dtype, tgt_length=input_shape[-1])
                # For decoding phase in generation, seq_length = 1, we don't need to add causal mask
                if input_shape[-1] > 1:
                    combined_attention_mask = _make_causal_mask(
                        input_shape,
                        past_key_values_length=past_key_values_length,
                    )
                    expanded_attn_mask = expanded_attn_mask & combined_attention_mask
            # [bsz, seq_len, seq_len] -> [bsz, 1, seq_len, seq_len]
            elif len(attention_mask.shape) == 3:
                expanded_attn_mask = attention_mask.unsqueeze(1).astype("bool")
            # if attention_mask is already 4-D, do nothing
            else:
                expanded_attn_mask = attention_mask
        else:
            expanded_attn_mask = _make_causal_mask(
                input_shape,
                past_key_values_length=past_key_values_length,
            )
        # Convert bool attention_mask to float attention mask, which will be added to attention_scores later
        expanded_attn_mask = paddle.where(expanded_attn_mask, 0.0, paddle.finfo(dtype).min).astype(dtype)
        return expanded_attn_mask

    def forward(
        self,
        input_ids: paddle.Tensor = None,
        attention_mask: Optional[paddle.Tensor] = None,
        position_ids: Optional[paddle.Tensor] = None,
        past_key_values: Optional[List[paddle.Tensor]] = None,
        inputs_embeds: Optional[paddle.Tensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        cache_position: Optional[paddle.Tensor] = None,
    ) -> Union[Tuple, BaseModelOutputWithPast]:
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError(
                "You cannot specify both input_ids and inputs_embeds at the same time, and must specify either one"
            )
        elif input_ids is not None:
            batch_size, seq_length = input_ids.shape
        elif inputs_embeds is not None:
            batch_size, seq_length, _ = inputs_embeds.shape
        else:
            raise ValueError("You have to specify either decoder_input_ids or decoder_inputs_embeds")

        if past_key_values is None:
            past_key_values = tuple([None] * len(self.layers))
        # NOTE: to make cache can be clear in-time
        past_key_values = list(past_key_values)

        seq_length_with_past = seq_length
        cache_length = 0
        if past_key_values[0] is not None:
            cache_length = past_key_values[0][0].shape[2]  # shape[1] in qwen2
            seq_length_with_past += cache_length

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids)

        # embed positions
        if attention_mask is None:
            # [bs, seq_len]
            attention_mask = paddle.ones((batch_size, seq_length_with_past), dtype=paddle.bool)

        attention_mask = self._prepare_decoder_attention_mask(
            attention_mask, (batch_size, seq_length), cache_length, inputs_embeds.dtype
        )  # [bs, 1, seq_len, seq_len]
        causal_mask = attention_mask

        if cache_position is None:
            past_seen_tokens = past_key_values[0][0].shape[2] if past_key_values[0] is not None else 0
            cache_position = paddle.arange(past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1])

        if position_ids is None:
            # the hard coded `3` is for temporal, height and width.
            position_ids = cache_position.reshape([1, 1, -1]).expand([3, inputs_embeds.shape[0], -1])

        hidden_states = inputs_embeds

        # decoder layers
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        next_decoder_cache = ()

        for idx, (decoder_layer) in enumerate(self.layers):
            if output_hidden_states:
                all_hidden_states += (hidden_states,)

            past_key_value = past_key_values[idx] if past_key_values is not None else None

            layer_outputs = decoder_layer(
                hidden_states,
                attention_mask=causal_mask,
                position_ids=position_ids,
                past_key_value=past_key_value,
                output_attentions=output_attentions,  # False
                use_cache=use_cache,  # True
                cache_position=cache_position,
            )

            # NOTE: clear outdate cache after it has been used for memory saving
            past_key_value = past_key_values[idx] = None

            hidden_states = layer_outputs[0]

            next_decoder_cache = next_decoder_cache + (layer_outputs[-1],) if use_cache else None

            if output_attentions:
                all_self_attns += (layer_outputs[1],)

        hidden_states = self.norm(hidden_states)

        # add hidden states from the last decoder layer
        if output_hidden_states:
            all_hidden_states += (hidden_states,)

        next_cache = next_decoder_cache if use_cache else None

        if not return_dict:
            return tuple(v for v in [hidden_states, next_cache, all_hidden_states, all_self_attns] if v is not None)
        return BaseModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=next_cache,
            hidden_states=all_hidden_states,
            attentions=all_self_attns,
        )


### 1.1.22 Qwen2LMHead

**Qwen2LMHead**

该类实现了一个语言建模头，用于将解码器的隐藏状态转换为词汇表上的分布（即输出概率）

**关键属性与方法**

__init__ 构造方法:


初始化模型的权重矩阵 self.weight。

**提供两种权重存储形式：**

转置形式 (transpose_y=True): 通常用于共享输入嵌入层的权重。

标准形式 (transpose_y=False): 用于独立定义输出层的权重。

**forward 方法:**

通过矩阵乘法，将解码器的隐藏状态映射到词汇表分布。

In [ ]:
class Qwen2LMHead(nn.Layer):
    def __init__(self, config, embedding_weights=None, transpose_y=False):
        super(Qwen2LMHead, self).__init__()
        self.config = config
        vocab_size = config.vocab_size

        self.transpose_y = transpose_y
        if transpose_y:
            # only for weight from embedding_weights
            if embedding_weights is not None:
                self.weight = embedding_weights
            else:
                self.weight = self.create_parameter(
                    shape=[vocab_size, config.hidden_size],
                    dtype=paddle.get_default_dtype(),
                )
        else:
            # for weight from model init
            self.weight = self.create_parameter(
                shape=[config.hidden_size, vocab_size],
                dtype=paddle.get_default_dtype(),
            )

    def forward(self, hidden_states):
        logits = paddle.matmul(hidden_states, self.weight, transpose_y=self.transpose_y)
        return logits

## **1.2 视觉编码器**

视觉编码器的作用是从图像中提取有意义的特征，并将其转换为适合语言模型的格式。Qwen2-VL 使用了视觉Transformer架构来处理图像数据。


**视觉编码器配置**:**Qwen2VLVisionConfig**

首先,**Qwen2VLVisionConfig** 类定义了视觉编码器的配置，它继承自 **PretrainedConfig** 类，允许加载预训练的模型配置。以下是该类的核心参数：

In [ ]:
class Qwen2VLVisionConfig(PretrainedConfig):
    model_type = "qwen2_vl"

    def __init__(self, depth=32, embed_dim=1280, hidden_size=3584, hidden_act="quick_gelu", mlp_ratio=4, num_heads=16, in_channels=3, patch_size=14, spatial_merge_size=2, temporal_patch_size=2, attn_implementation="eager", **kwargs):
        super().__init__(**kwargs)
        self.depth = depth
        self.embed_dim = embed_dim
        self.hidden_size = hidden_size
        self.hidden_act = hidden_act
        self.mlp_ratio = mlp_ratio
        self.num_heads = num_heads
        self.in_channels = in_channels
        self.patch_size = patch_size
        self.spatial_merge_size = spatial_merge_size
        self.temporal_patch_size = temporal_patch_size
        self.attn_implementation = attn_implementation


**关键参数解析:**

**patch_size：**表示图像切割成小块的大小。Qwen2-VL 采用了视觉 Transformer（ViT）架构，因此将图像分为多个固定大小的 patch，每个 patch 会被映射为一个向量。

**depth：**Transformer 的层数，决定了视觉编码器的深度。

**embed_dim：**每个图像 patch 的嵌入维度。

**hidden_size：**隐藏层的大小，决定了模型处理信息的能力。

**num_heads：**注意力头的数量，影响 Transformer 注意力机制的并行性。

**attn_implementation：**表示使用的注意力实现方式，eager 或 flash，前者为传统的计算方式，后者使用 flash attention 优化计算速度。

**视觉编码器的实现：**


视觉编码器的实现包含了图像的切分、嵌入生成和位置编码，具体代码如下：

In [ ]:
class Qwen2VisionTransformerPretrainedModel(PreTrainedModel):
    def __init__(self, config: Qwen2VLVisionConfig):
        super().__init__(config)
        self.config = config

        # 图像分块并映射到嵌入
        self.patch_embed = PatchEmbed(config.in_channels, config.embed_dim, config.patch_size)

        # 视觉位置编码
        self.position_embeddings = VisionRotaryEmbedding(config.embed_dim, config.hidden_size)

        # Transformer Block
        self.blocks = nn.ModuleList([Qwen2VLVisionBlock(config) for _ in range(config.depth)])

        # 生成图像的最终嵌入表示
        self.layernorm = nn.LayerNorm(config.hidden_size)
        self.pooler = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        # 生成图像的patch嵌入
        x = self.patch_embed(x)
        x = self.position_embeddings(x)

        # 通过Transformer层
        for block in self.blocks:
            x = block(x)

        # 最后的池化和层归一化
        x = self.layernorm(x)
        x = self.pooler(x)

        return x


**代码解析:**

**PatchEmbed：**负责将图像分成多个 **patch** 并将每个**patch**转化为一个向量。这样就将二维图像转换为一维序列，类似于文本中的 token。

**VisionRotaryEmbedding：**这部分是位置编码模块，用于为每个**patch**添加位置信息，使模型能够处理不同位置的图像。

**Qwen2VLVisionBlock：**每个**VisionBlock**包含多头注意力机制和前馈网络**（MLP）**。它是视觉Transformer的核心模块。


## **1.3 语言模型**

Qwen2-VL 的语言模型部分主要处理文本输入，并生成对应的语言输出或进行推理。语言模型部分使用 Transformer 解码器架构。


**语言模型配置**:**Qwen2VLConfig**

**Qwen2VLConfig**类提供了整个语言模型的配置，它继承自**PretrainedConfig**，包括以下参数：

In [ ]:
class Qwen2VLConfig(PretrainedConfig):
    model_type = "qwen2_vl"
    def __init__(self, vocab_size=152064, hidden_size=8192, intermediate_size=29568, num_hidden_layers=80, num_attention_heads=64, num_key_value_heads=8, hidden_act="silu", max_position_embeddings=32768, initializer_range=0.02, rms_norm_eps=1e-05, use_cache=True, tie_word_embeddings=False, rope_theta=1000000.0, use_sliding_window=False, sliding_window=4096, max_window_layers=80, attention_dropout=0.0, vision_config=None, rope_scaling=None, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_key_value_heads = num_key_value_heads
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.use_cache = use_cache
        self.tie_word_embeddings = tie_word_embeddings
        self.rope_theta = rope_theta
        self.use_sliding_window = use_sliding_window
        self.sliding_window = sliding_window
        self.max_window_layers = max_window_layers
        self.attention_dropout = attention_dropout
        self.vision_config = vision_config
        self.rope_scaling = rope_scaling


**语言模型的实现**

语言模型的实现基于 Transformer 解码器，使用多层自注意力（Self-Attention）和前馈网络（Feed-Forward Network）。

In [ ]:
class Qwen2VLForConditionalGeneration(PreTrainedModel):
    def __init__(self, config: Qwen2VLConfig):
        super().__init__(config)
        self.config = config

        # 词嵌入层
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size)

        # Transformer 解码器
        self.decoder = nn.ModuleList([Qwen2VLDecoderLayer(config) for _ in range(config.num_hidden_layers)])

        # 输出层
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

    def forward(self, input_ids):
        # 嵌入并处理位置编码
        x = self.embedding(input_ids)

        # 通过 Transformer 解码器
        for layer in self.decoder:
            x = layer(x)

        # 输出生成的 logits
        logits = self.lm_head(x)

        return logits


**代码解析:**

**self.embedding：**将文本 token 转换为词嵌入（embedding），这些嵌入表示文本中的语义。

**Qwen2VLDecoderLayer：**每一层包含自注意力机制、前馈网络（MLP）和层归一化。它是解码器的核心模块。

**lm_head：**输出层，将隐藏层的输出映射回词汇表大小，以生成每个位置的 token 概率分布。



## **1.4 多模态融合**

Qwen2-VL 模型的独特之处在于，它能够处理并融合视觉和语言信息。视觉输入通过视觉编码器处理后与语言输入的特征进行融合，从而完成跨模态任务。

**融合方式：**

In [ ]:
class Qwen2VLForConditionalGeneration(PreTrainedModel):
    def forward(self, input_ids, visual_features):
        # 将视觉特征和文本输入拼接
        x = torch.cat([input_ids, visual_features], dim=1)

        # 通过 Transformer 解码器处理
        for layer in self.decoder:
            x = layer(x)

        # 输出生成的 logits
        logits = self.lm_head(x)

        return logits


**代码解析:**

**visual_features：**这是经过视觉编码器处理后的图像特征。它们与文本输入的 input_ids 一起输入到解码器中。

**torch.cat：**视觉特征和文本输入拼接在一起，形成多模态输入，之后通过 Transformer 解码器进行处理。



